In [2]:
from _linear import LinearRegression
import _dgp as dg
from _metrics import mse_score, CrossValidation, accuracy_score,f1_score
from  _logistic import LogisticRegression
import numpy as np
import pandas as pd

## **Table of Contents**

- [Linear Regression](#linear-regression)
  - [Generate data and fit first model](#generate-data-and-fit-first-model)
  - [Evaluate model performance](#evaluate-model-performance)
  - [Automatic selection of variables](#automatic-selection-of-variables) 
  - [Drawbacks and to do list](#drawbacks-and-to-do-list)

- [Logistic Regression](#logistic-regression)
  - [Work with excel data (5 variables in total)](#work-with-excel-data-5-variables-in-total)
  - [Evaluate model performance](#evaluate-model-performance-logistic-regression)
  - [Automatic selection of variables](#automatic-selection-of-variables-logistic-regression)
  - [Generate 50 variables and autoselection](#Generate-50-variables-and-autoselection)



- [Logistic Regression OVR](#Logistic-Regression-OVR)
  - [Fit and predict model](#Fit-and-predict-model)
  - [Evaluate model performance](#Evaluate-model-performance-Log-OVR)
  - [Automatic selection of variables] incoming



- [Logistic Regression Softmax](#Logistic-Regression-Softmax)
  - [Fit and predict model](#Fit-and-predict-model-Log-SFTMX)
  - [Evaluate model performance](#Evaluate-model-performance-Log-SFTMX)
  - [Automatic selection of variables] incoming



## **Linear Regression**


### **Generate data and fit first model**

Generate data with no correlation with only 5 true variables among 50 variables

In [2]:
x,y=dg.gen(type=1)
#get matrix size to understand what is the data begind
print(x.shape)
print(y.shape)

(500, 50)
(500,)


Run linear regression and get B parameter

In [3]:
#use Newton Raphson optimisation which is the best optimiser for parametric models
regression=LinearRegression(solver="nr")
regression.fit(x,y)

algorithm did  converge under 100 iterations (at 2 iterations)


array([-3.93099398e-03,  2.49595541e+00,  1.30241480e+00,  5.00668710e-01,
       -4.95691678e-01, -3.39506857e+00, -3.41591621e-03, -6.08881099e-04,
       -1.65135863e-03,  4.64044304e-03, -2.73076139e-03, -6.06228269e-04,
       -3.15465712e-03, -1.79282286e-03,  2.43467778e-03,  5.82342314e-03,
       -4.70361961e-03, -7.39833408e-03, -2.42082552e-04,  9.77331694e-03,
       -3.28885097e-03,  4.60316476e-03, -3.80177844e-03,  1.66342373e-04,
        6.30058320e-03, -1.49051243e-03, -3.79969229e-03,  8.38710144e-04,
        6.27680104e-03,  3.32856619e-04, -5.19931099e-03, -1.00368689e-03,
       -3.07440798e-03, -2.65562808e-03, -4.58757740e-04, -7.01413457e-03,
       -6.47010425e-04, -8.23903640e-03, -8.92327582e-03,  1.32614184e-03,
        3.13712693e-03, -3.59975518e-03, -4.11818560e-04,  8.23935282e-03,
       -1.78937922e-03,  2.57606393e-03, -1.92287378e-03, -1.50376437e-03,
        8.11343362e-03, -1.74188516e-03,  1.82320671e-03])

As we used NR optimisation, it converged very fast, only in 2 iterations

if We want to interpret Linear regression results, we can do so by typing this command:

In [4]:
regression.get_inference().head()

,params,std,t value,p value
0,-0.003931,0.004469,-0.879564,0.3796
1,2.495955,0.004348,574.070347,0.0000
2,1.302415,0.004321,301.449762,0.0000
3,0.500669,0.004425,113.136260,0.0000
4,-0.495692,0.004625,-107.185529,0.0000


### **Evaluate model performance**

We can evaluate model performance by looking at Mean squared error

In [17]:
mse_score(y,regression.predict(x))

0.007461950038721623

However, the best way to evaluate model performance remains Cross Validation for several reasons

In [22]:
list_of_mse=CrossValidation(Class_algorithm=regression,x=x,y=y,metrics_function=mse_score,nb_k_fold=6)
print(list_of_mse)

algorithm did  converge under 100 iterations (at 2 iterations)
algorithm did  converge under 100 iterations (at 2 iterations)
algorithm did  converge under 100 iterations (at 2 iterations)
algorithm did  converge under 100 iterations (at 2 iterations)
algorithm did  converge under 100 iterations (at 2 iterations)
algorithm did  converge under 100 iterations (at 2 iterations)
algorithm did  converge under 100 iterations (at 2 iterations)
[0.008366650090492278, 0.008926335388529164, 0.009450567718380496, 0.011295040543736782, 0.009873535736372823, 0.009144819190386013, 0.0031046842380932516]


we can average these results to get average model performance

In [23]:
np.mean(list_of_mse)

0.008594518986570114

### **Automatic selection of variables**

**However, it is not a good practice to perform linear regression using all variables.** In order to obtain the best subset, several strategies can be adopted:

- **VIF models:** These identify and drop collinear variables to avoid multicollinearity issues.
- **Lasso regression:** This technique can be used for feature selection by penalizing the absolute size of the regression coefficients.
- **Forward/Backward/Stepwise selection:** These are iterative methods for feature selection, where variables are added or removed based on their impact on model performance.

In this section we will use last suggestion


In [6]:
col_index=regression.autoselection("forward","BIC_ll",print_message=False)
col_index

array([4, 0, 1, 2, 3])

and we get the following columns of X Matrix , first 5 variables that are also 5 true variables that we defined in dgp

In [7]:
x[:,col_index]

array([[ 1.38867355,  1.67340652, -0.92325656,  0.48536469, -0.99374157],
       [ 0.57639269,  1.30833597, -1.90600557,  0.099619  , -0.30602642],
       [-1.22795098,  1.24972809,  1.27848344,  1.51070181,  0.3392137 ],
       ...,
       [-1.93073004,  0.41164028, -0.01779019,  0.10208403,  0.40405628],
       [ 0.44262482, -0.32552216,  0.80148378,  0.81213097,  1.19487167],
       [ 0.0796678 , -0.48110001,  1.21688749, -2.83203312, -0.25707275]])

In [8]:
col_index=regression.autoselection("stepwise","BIC_ll",print_message=False)
col_index

array([2, 0, 4, 1, 3])

In [9]:
col_index=regression.autoselection("backward","BIC_ll",print_message=False)
col_index

array([0, 1, 2, 3, 4])

### **Drawbacks and to do list**

**In this section, we discussed the machine learning approach of estimating the model parameters of Linear Regression.**

<span style="color:red"> However, we did not analyze the validity tests of normality and homoscedasticity of residuals, endogenous variables, etc. This section will be developed soon.</span>









## **Logistic Regression**

### **Work with excel data (5 variables in total)**

Load data and explore 

In [3]:
table=pd.read_csv("data/exampleLR.csv")

In [4]:
table.head()

,Unnamed: 0,V1,V2,V3,V4,V5,y
0,0,-0.560476,-0.601893,-0.995799,-0.820987,-0.511604,0
1,1,-0.230177,-0.993699,-1.039955,-0.307257,0.236938,0
2,2,1.558708,1.026785,-0.017980,-0.902098,-0.541589,1
3,3,0.070508,0.751061,-0.132175,0.627069,1.219228,0
4,4,0.129288,-1.509167,-2.549343,1.120355,0.174136,0


Always convert data to numpy array before starting the work

In [5]:
x,y=np.array(table.drop(["y","Unnamed: 0"],axis=1)),np.array(table["y"])

**Are classes balanced?**

- If yes, the use of accuracy is enough (our case).
- If not:
  - Use other metrics such as precision, recall, F1 score, etc.
  - Do over/undersampling.



In [32]:
table["y"].value_counts()

y
0    253
1    247
Name: count, dtype: int64

Try Gradient Descent algorithm

In [6]:
model=LogisticRegression(solver="gd")
model.fit(x,y)

algorithm did not converge under 100 iterations,so the calculated parameter is biased


array([-0.1248317 , -0.98031107,  1.93586459,  0.89773156,  0.7761705 ,
       -1.07133784])

**Algorithm could not converge:**
If you have this message, **NEVER** proceed to further steps as 
the estimated parameter is biased. 

Therefore, try other hyperparameters to aim  convergence.


In [7]:
model1=LogisticRegression(solver="gd",learning_rate=0.004,max_iteration=300)
model1.fit(x,y)

algorithm did  converge under 300 iterations (at 79 iterations)


array([-0.14211795, -1.10847242,  2.18459166,  1.02655283,  0.87139786,
       -1.20943036])

Try stochastic gradient descent with 68 as size of mini batch to see the performance

In [68]:
model2=LogisticRegression(solver="sgd",learning_rate=0.001,max_iteration=300,mini_batch_size=68,add_intercept=False)
model2.fit(x,y)

algorithm did  converge under 2400 iterations (at 1080 iterations)


array([-1.03254695,  2.03007399,  0.9448015 ,  0.80938114, -1.11919829])

With new learning rate algorithm converged so we will use results of model1 to make predictions

In [72]:
model1.predict(x)[0:10]

array([0, 0, 1, 1, 0, 1, 1, 0, 1, 1])

as well as get parameters significance

In [76]:
print(model1.get_inference())
print(model1.criteria)

     params       std   t value  p value
0 -0.142118  0.133005 -1.068517   0.2858
1 -1.108472  0.159209 -6.962360   0.0000
2  2.184592  0.222661  9.811274   0.0000
3  1.026553  0.163791  6.267456   0.0000
4  0.871398  0.144580  6.027095   0.0000
5 -1.209430  0.173253 -6.980720   0.0000
{'LL': -177.40450034974413, 'AIC_ll': 366.80900069948825, 'BIC_ll': 392.0966492900214}


### **Evaluate model performance** (Logistic Regression)

Evaluate model with accuracy and f1score 

In [79]:
CrossValidation(Class_algorithm=model1,x=x,y=y,metrics_function=accuracy_score,nb_k_fold=4)

algorithm did  converge under 300 iterations (at 122 iterations)
algorithm did  converge under 300 iterations (at 86 iterations)
algorithm did  converge under 300 iterations (at 101 iterations)
algorithm did  converge under 300 iterations (at 95 iterations)


[0.8, 0.904, 0.816, 0.864]

In [80]:
CrossValidation(Class_algorithm=model1,x=x,y=y,metrics_function=f1_score,nb_k_fold=4)

algorithm did  converge under 300 iterations (at 114 iterations)
algorithm did  converge under 300 iterations (at 94 iterations)
algorithm did  converge under 300 iterations (at 99 iterations)
algorithm did  converge under 300 iterations (at 96 iterations)


[0.7839999999999999, 0.8925619834710744, 0.832, 0.8480000000000001]

### **Automatic selection of variables** (Logistic Regression)

In [8]:
index_cols=model1.autoselection("forward","BIC_ll",print_message=False)
index_cols

array([1, 4, 0, 2, 3])

As you see in this case we get 5 variables which are 5 true variables , but this is also a generated data from excel file with 5 variables

We can see how this selection performs on another generation that will contain 5 true variables and 45 false variables

### **Generate 50 variables and autoselection**

In [9]:
x,y=dg.gen(1,"logistic")

In [10]:
model_new=LogisticRegression(solver="nr")
model_new.fit(x,y)

algorithm did  converge under 100 iterations (at 8 iterations)


array([-2.35615428e-02,  2.96272801e+00,  1.63301139e+00,  6.46517737e-01,
       -6.47987762e-01, -4.24920907e+00, -3.62698505e-02,  7.88733299e-02,
        1.30400295e-01,  2.01457804e-01,  4.67478709e-02, -4.21751352e-02,
       -8.32915090e-02, -4.49868261e-02, -7.64936897e-02,  3.19617708e-01,
        2.60727696e-01, -3.09929400e-01,  2.22953074e-02,  1.49686161e-01,
        4.87909799e-02, -1.29314407e-01,  2.01636257e-01, -3.94992832e-01,
       -1.92925329e-01,  1.46761102e-01,  2.71069222e-01, -2.41546209e-01,
       -1.52664588e-01, -2.11520810e-01,  4.13880540e-01,  1.23104652e-01,
       -2.09400301e-01, -1.27855414e-01,  8.25100569e-02, -1.40334849e-01,
        8.23030185e-04, -2.02431627e-01,  9.66541365e-02, -7.14232283e-02,
       -2.84819340e-02, -1.61517332e-01, -2.16718932e-02,  1.06732851e-01,
        1.24860888e-01,  5.87703507e-02,  1.21029240e-01, -1.46685921e-01,
       -1.02176563e-01, -1.18832302e-01, -1.84258521e-01])

In [12]:
model_new.autoselection("forward","BIC_ll",print_message=False)

array([4, 0, 1, 2, 3])

In [13]:
model_new.autoselection("stepwise","BIC_ll",print_message=False)

array([3, 2, 1, 4, 0])

In [14]:
model_new.autoselection("backward","BIC_ll",print_message=False)

array([0, 1, 2, 3, 4])

Despite having 50 variables in the dataset, the model successfully identified only 5 true variables. All 3 models gave same results which is a normal behaviour.

## **Logistic Regression OVR**

### **Fit and predict model**

Load data

In [85]:
table=pd.read_csv("data/exampleLRMulti.csv")

Get rid of unnamed columns

In [87]:
table.drop(table.columns[table.columns.str.contains('unnamed',case = False)],axis = 1, inplace = True)

One more time classes are distributed equally, so we can estimate model performance with accuracy score

In [91]:
table["column_Y"].value_counts()

column_Y
1    336
0    333
2    331
Name: count, dtype: int64

In [92]:
x,y=np.array(table.drop(["column_Y"],axis=1)),np.array(table["column_Y"])

Estimate Multiclass Logistic Regression with One vs Rest algorithm and Newton Raphson optimisation 

Since it's an OVR algorithm, it will run model 3 times with 3 columns of estimated parameter

In [93]:
modelOVR=LogisticRegression(solver="nr",multiclass="ovr")
modelOVR.fit(x,y)

algorithm did  converge under 100 iterations (at 5 iterations)
algorithm did  converge under 100 iterations (at 5 iterations)
algorithm did  converge under 100 iterations (at 5 iterations)


array([[-1.01498187, -1.45620846, -0.78718645],
       [ 0.34446018,  0.56751748, -0.79115877],
       [ 0.22977187, -0.65556071,  0.56300868],
       [ 0.05302252, -0.03242775, -0.03517929],
       [ 0.06803143,  0.15324563, -0.1736295 ],
       [-0.44905806,  0.79972167, -0.42289584],
       [ 0.02949896, -0.09727438,  0.07434896],
       [-0.64703364,  0.23914131,  0.39520493],
       [-0.03647586,  0.15863365, -0.09606587],
       [ 0.6808661 , -0.45149573, -0.23001672],
       [-0.01179362, -0.05067621,  0.03836354]])

We can also predict. Predictions are made always in the ascending order: 

    -first column will be 0 class
    -second column will be 1 class
    -third column will be 3 class


In [100]:
predictionsOVR=modelOVR.predict(x)
predictionsOVR 

array([[0., 1., 0.],
       [0., 1., 0.],
       [0., 0., 1.],
       ...,
       [0., 0., 1.],
       [0., 1., 0.],
       [0., 0., 1.]])

We will compare these predictions to true one hot encoded labels

In [101]:
modelOVR.true_labels_matrix

array([[0, 1, 0],
       [1, 0, 0],
       [0, 0, 1],
       ...,
       [0, 0, 1],
       [1, 0, 0],
       [0, 0, 1]])

Of course, we can always get probability (for OVR it is not normalised, but for predicted values it is normalised!!)

In [102]:
modelOVR.proba

array([[0.14660444, 0.94592762, 0.01074276],
       [0.26867955, 0.36961135, 0.19349834],
       [0.0519826 , 0.32131164, 0.59426132],
       ...,
       [0.02941952, 0.15974979, 0.77393711],
       [0.26880729, 0.33355668, 0.22603021],
       [0.03407864, 0.33402281, 0.64490065]])

### **Evaluate model performance** (Log OVR)

We can evaluate accuracy

In [104]:
accuracy_score(predictionsOVR, modelOVR.true_labels_matrix)

0.695

But we want more intelligent evaluation, so we use cross validation 

In [107]:
acc_list=CrossValidation(Class_algorithm=modelOVR,x=x,y=modelOVR.true_labels_matrix,metrics_function=accuracy_score,nb_k_fold=8)

algorithm did  converge under 100 iterations (at 5 iterations)
algorithm did  converge under 100 iterations (at 5 iterations)
algorithm did  converge under 100 iterations (at 5 iterations)
algorithm did  converge under 100 iterations (at 5 iterations)
algorithm did  converge under 100 iterations (at 5 iterations)
algorithm did  converge under 100 iterations (at 5 iterations)
algorithm did  converge under 100 iterations (at 5 iterations)
algorithm did  converge under 100 iterations (at 5 iterations)
algorithm did  converge under 100 iterations (at 5 iterations)
algorithm did  converge under 100 iterations (at 5 iterations)
algorithm did  converge under 100 iterations (at 5 iterations)
algorithm did  converge under 100 iterations (at 5 iterations)
algorithm did  converge under 100 iterations (at 5 iterations)
algorithm did  converge under 100 iterations (at 5 iterations)
algorithm did  converge under 100 iterations (at 5 iterations)
algorithm did  converge under 100 iterations (at 5 iter

By averaging we get more "robust" accuracy 

In [108]:
print(acc_list)
np.mean(acc_list)

[0.68, 0.632, 0.68, 0.704, 0.696, 0.696, 0.624, 0.672]


0.6729999999999999

## **Logistic Regression Softmax**

**Now, let's see another way of solving multicase without running P independent classifiers-> logistic softmax**

### **Fit and predict model** (Log SFTMX)

In [115]:
modelSFT=LogisticRegression(solver="gd",multiclass="softmax")#dont use nr for the moment
modelSFT.fit(x,y)

algorithm did  converge under 100 iterations (at 50 iterations)


array([[ 0.04194506, -0.27091767,  0.22897262],
       [ 0.23740402,  0.34859258, -0.58599659],
       [ 0.08908508, -0.4494452 ,  0.36036012],
       [ 0.0417117 , -0.02254157, -0.01917013],
       [ 0.03704339,  0.08616513, -0.12320852],
       [-0.27315999,  0.56539992, -0.29223993],
       [ 0.01838198, -0.07586092,  0.05747895],
       [-0.47306334,  0.21607495,  0.2569884 ],
       [-0.03294754,  0.10874983, -0.07580229],
       [ 0.48652451, -0.33256412, -0.15396038],
       [-0.00460555, -0.02954451,  0.03415006]])

We can compare them to modelOVR parameters

In [112]:
modelOVR.params

array([[-1.01498187, -1.45620846, -0.78718645],
       [ 0.34446018,  0.56751748, -0.79115877],
       [ 0.22977187, -0.65556071,  0.56300868],
       [ 0.05302252, -0.03242775, -0.03517929],
       [ 0.06803143,  0.15324563, -0.1736295 ],
       [-0.44905806,  0.79972167, -0.42289584],
       [ 0.02949896, -0.09727438,  0.07434896],
       [-0.64703364,  0.23914131,  0.39520493],
       [-0.03647586,  0.15863365, -0.09606587],
       [ 0.6808661 , -0.45149573, -0.23001672],
       [-0.01179362, -0.05067621,  0.03836354]])

also predict

In [113]:
modelSFT.predict(x)

array([[0., 1., 0.],
       [0., 1., 0.],
       [0., 0., 1.],
       ...,
       [0., 0., 1.],
       [0., 1., 0.],
       [0., 0., 1.]])

### **Evaluate model performance** (Log SFTMX)

In [116]:
meansftmx=CrossValidation(Class_algorithm=modelSFT,x=x,y=modelSFT.true_labels_matrix,metrics_function=accuracy_score,nb_k_fold=4)
np.mean(meansftmx)

algorithm did  converge under 100 iterations (at 65 iterations)
algorithm did  converge under 100 iterations (at 66 iterations)
algorithm did  converge under 100 iterations (at 56 iterations)
algorithm did  converge under 100 iterations (at 62 iterations)


0.671

Further steps:

 -introduce Newton Raphson for softmax regression

 -explore further inference statistics ( log odds, inference for multinomial logistic regressions)

 -do automatic variable selection ( stepwise, backward..., lasso..)
 
 -do real project using all these developped algorithms



In [118]:
modelSFT=LogisticRegression(solver="nr",multiclass="softmax")#dont use nr for the moment
modelSFT.fit(x,y)

ValueError: not done yet,incoming